In [1]:
from kiteconnect import KiteConnect

In [2]:
import logging
import requests

In [3]:
import pandas as pd
from time import time, sleep
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [5]:
api_key = "f5ean4aaxq3wv9n3"
client_id = "ED5107"
api_secret = "s1kbs5y654zhbpqgd2rvslqc6shx9yh8"

In [6]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.trade/connect/login?api_key=f5ean4aaxq3wv9n3&v=3


In [7]:
# Get the Request Token from the redirect URL

request_token = "NDXIonWGnx63UBU3eQPtksuvWe31laSP"

In [8]:
#data = kite.generate_session(request_token, api_secret=api_secret)
#access_token = data["access_token"]
access_token = "aBopvJ6255CQwQAbmPDDuPeX383so5CF"
kite.set_access_token(access_token)
print(access_token)

aBopvJ6255CQwQAbmPDDuPeX383so5CF


In [9]:
print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/1.1" 200 None


{'user_id': 'ED5107', 'user_type': 'individual', 'email': 'ramanathannalini@gmail.com', 'user_name': 'Nalini Ramanathan', 'user_shortname': 'Nalini', 'broker': 'ZERODHA', 'exchanges': ['BSE', 'BFO', 'CDS', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': '', 'meta': {'demat_consent': 'consent'}}


In [10]:
# Get Instrument Tokens

def get_instrument_token(tradingsymbol, instrument_type, exchange):
    instruments = pd.DataFrame(kite.instruments())
    instrument_token = instruments[(instruments['tradingsymbol']==tradingsymbol) & (instruments['instrument_type']==instrument_type) & (instruments['exchange']==exchange)]['instrument_token'].to_list()[0]
    return str(instrument_token)

In [11]:
#Get Historical Data
def get_historical_data(instrument_token, from_date, to_date, interval):
    return pd.DataFrame(kite.historical_data(instrument_token=instrument_token,
                                    from_date=from_date, 
                                   to_date=to_date,
                                   interval=interval))

#Get Exponential Moving Average
def get_exponential_moving_avg(timeseries, span):
    """
    adjust=False specifies that we are interested in the recursive calculation mode.
    """
    timeseries[str(span)+'d_EMAvg'] = timeseries.iloc[:,0].ewm(span=span, adjust=False).mean()
    return timeseries

#Stochastic Momentum Indicator
def get_smi(hist_data, k_period = 10, k_smooth_period = 3, d_smooth_period = 3, d_period = 10):
    close = pd.DataFrame({'date':[datetime.strftime(d, "%Y-%m-%d %H:%M:%S") for d in list(hist_data['date'])],'close':list(hist_data['close'])})
    highest_high = pd.DataFrame(data=list(hist_data['high']), columns=['highest_high']).rolling(k_period, min_periods=1).max()
    lowest_low = pd.DataFrame(data=list(hist_data['low']), columns=['lowest_low']).rolling(k_period, min_periods=1).min()
    mean_hl = pd.DataFrame(data=(highest_high['highest_high']+lowest_low['lowest_low'])*0.5, columns=['mean_hl'])
    hh_minus_ll = pd.DataFrame(data=(highest_high['highest_high']-lowest_low['lowest_low']), columns=['hh_minus_ll'])
    close_minus_mean_hl = pd.DataFrame(data=(close['close']-mean_hl['mean_hl']), columns=['close_minus_mean_hl'])
    emavg_cmmhl = pd.DataFrame(data=list(close_minus_mean_hl['close_minus_mean_hl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_cmmhl'])
    emavg_emavg_cmmhl = pd.DataFrame(data=list(emavg_cmmhl['emavg_cmmhl'].ewm(span=k_smooth_period, adjust=False).mean()), columns=['emavg_emavg_cmmhl'])
    emavg_hhmll = pd.DataFrame(data=list(hh_minus_ll['hh_minus_ll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_hhmll'])
    emavg_emavg_hhmll = pd.DataFrame(data=list(emavg_hhmll['emavg_hhmll'].ewm(span=d_smooth_period, adjust=False).mean()), columns=['emavg_emavg_hhmll'])
    smi = pd.DataFrame(data=100*(emavg_emavg_cmmhl['emavg_emavg_cmmhl']/(emavg_emavg_hhmll['emavg_emavg_hhmll']/2)), columns=['smi'])
    signal = pd.DataFrame(data=list(smi['smi'].ewm(span=d_period, adjust=False).mean()), columns=['signal'])
    smi_df = pd.concat([close, smi, signal],axis=1)
    smi_df.set_index('date', drop=True, inplace=True)
    return smi_df

# Zerodha Testing LIVE Deployment TradeLog
def place_trade(trade_log, current_time, price, open_price, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    if price>open_price:
        candle_color = 'green'
    else:
        candle_color = 'red'
    print(current_time, current_transaction_type, price, current_status)
    trade_log.loc[len(trade_log)] = [current_time, open_price, price, candle_color, current_transaction_type, current_status]
    return trade_log, price, current_transaction_type, current_status

# Pre-Flight Checks
#### WARNING: Don't forget to update the date and time in 'deply_till' variable


In [12]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20AUGFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
candle_size = '5minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=25, hour=15 , minute=30 , second=0)

In [13]:
instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 910022


In [14]:
instoken

'11348226'

In [16]:
kite.quote(exchange+":"+tradingsymbol)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None


{'NFO:BANKNIFTY20AUGFUT': {'instrument_token': 11348226,
  'timestamp': datetime.datetime(2020, 8, 24, 11, 15, 9),
  'last_trade_time': datetime.datetime(2020, 8, 24, 11, 15, 9),
  'last_price': 22671,
  'last_quantity': 50,
  'buy_quantity': 199725,
  'sell_quantity': 144125,
  'volume': 1691050,
  'average_price': 22597.42,
  'oi': 1537225,
  'oi_day_high': 1537450,
  'oi_day_low': 1297275,
  'net_change': 0,
  'lower_circuit_limit': 20054.55,
  'upper_circuit_limit': 24511.1,
  'ohlc': {'open': 22327.25, 'high': 22692, 'low': 22327.2, 'close': 22282.8},
  'depth': {'buy': [{'price': 22668.3, 'quantity': 25, 'orders': 1},
    {'price': 22668.25, 'quantity': 50, 'orders': 1},
    {'price': 22668.2, 'quantity': 25, 'orders': 1},
    {'price': 22668, 'quantity': 200, 'orders': 3},
    {'price': 22667.95, 'quantity': 50, 'orders': 1}],
   'sell': [{'price': 22671, 'quantity': 25, 'orders': 1},
    {'price': 22673.3, 'quantity': 25, 'orders': 1},
    {'price': 22673.85, 'quantity': 50, 'o

In [15]:
while True:
    print(kite.quote(exchange+":"+tradingsymbol)[exchange+":"+tradingsymbol]['last_price'])
    sleep(1)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


22674.95


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


22672.1


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


...

DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


22670.4


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


22671.2


KeyboardInterrupt: 

In [35]:
candle = 3
first_candle_timestamp = datetime(2020, 8, 25, 14, 54)
candle_data = pd.DataFrame(columns=['Candle','Open','High','Low','Close','Color'])
latest_candle = pd.DataFrame(columns=['price'])

while datetime.strftime(datetime.now(), "%H:%M")!=datetime.strftime(deploy_till-timedelta(minutes=1), "%H:%M"):
    for _ in range(5):
        try:
            quote = kite.quote(exchange+":"+tradingsymbol)[exchange+":"+tradingsymbol]
            break
        except ReadTimeout:
            sleep(5)
            pass
    latest_price = quote['last_price']
    if datetime.now().minute>=first_candle_timestamp.minute and datetime.now().minute<(first_candle_timestamp+timedelta(minutes=candle)).minute:
        latest_candle.loc[len(latest_candle)] = latest_price
    elif (first_candle_timestamp+timedelta(minutes=candle)).minute==0 and datetime.now().hour<(first_candle_timestamp+timedelta(minutes=candle)).hour:
        latest_candle.loc[len(latest_candle)] = latest_price
    else:
        price_list = list(latest_candle['price'])
        if price_list[0]<price_list[-1]:
            color = 'green'
        else:
            color = 'red'
        candle_data.loc[len(candle_data)] = [first_candle_timestamp, price_list[0], max(price_list), min(price_list), price_list[-1], color]
        first_candle_timestamp = first_candle_timestamp+timedelta(minutes=candle)
        latest_candle = pd.DataFrame(columns=['price'])
    sleep(1)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ABANKNIFTY20AUGFUT HTTP/1.1" 200 None
DEBUG:urllib3.connec

In [36]:
candle_data

,Candle,Open,High,Low,Close,Color
0,2020-08-25 14:54:00,23001.75,23020.00,22998.35,23010.00,green
1,2020-08-25 14:57:00,23010.00,23050.00,23010.00,23037.20,green
2,2020-08-25 15:00:00,23037.85,23103.60,23037.85,23072.35,green
3,2020-08-25 15:03:00,23075.00,23084.15,23056.75,23070.00,red
4,2020-08-25 15:06:00,23067.30,23094.95,23061.30,23079.95,green
5,2020-08-25 15:09:00,23072.80,23075.35,23054.00,23062.55,red
6,2020-08-25 15:12:00,23065.00,23074.10,23064.70,23070.00,green
7,2020-08-25 15:15:00,23070.00,23080.00,23065.05,23070.25,green
8,2020-08-25 15:18:00,23074.90,23088.65,23073.20,23080.50,green
9,2020-08-25 15:21:00,23081.00,23090.75,23075.20,23090.75,green


In [ ]:
candle = 3
first_candle_timestamp = datetime(2020, 8, 26, 9, 15)

next_transaction_time = first_candle_timestamp + timedelta(minutes=candle) - timedelta(seconds=5)
trade_log = pd.DataFrame(columns=['time','open_price','price','candle_color','transaction_type','status'])
candle_data = pd.DataFrame(columns=['Candle','Open','High','Low','Close','Color'])
latest_candle = pd.DataFrame(columns=['price'])
current_transaction_type = None
current_status = 'closed'
pnl = 0
n=1

while datetime.strftime(datetime.now(), "%H:%M")!=datetime.strftime(deploy_till-timedelta(minutes=1), "%H:%M"):
    sleep(1)
    timestamp = datetime.now()
    
    for _ in range(5):
        try:
            quote = kite.quote(exchange+":"+tradingsymbol)[exchange+":"+tradingsymbol]
            break
        except ReadTimeout:
            sleep(5)
            pass

    price = quote['last_price']
    if timestamp.minute>=first_candle_timestamp.minute and timestamp.minute<(first_candle_timestamp+timedelta(minutes=candle)).minute:
        latest_candle.loc[len(latest_candle)] = price
    elif (first_candle_timestamp+timedelta(minutes=candle)).minute==0 and timestamp.hour<(first_candle_timestamp+timedelta(minutes=candle)).hour:
        latest_candle.loc[len(latest_candle)] = price
    
    if datetime.strftime(timestamp, "%H:%M")==datetime.strftime(deploy_till-timedelta(minutes=2), "%H:%M"):
        if current_status=='open':
            if current_transaction_type=='buy':
                transaction_type='sell'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
                break
            if current_transaction_type=='sell':
                transaction_type='buy'
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
                break
        else:
            break

    if current_transaction_type == 'buy':
        pnl = price - transaction_price
    elif current_transaction_type == 'sell':
        pnl = transaction_price - price

    if pnl < -50:
        if current_transaction_type == 'buy':
            transaction_type = 'sell'
            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
            current_transaction_type = None
        elif current_transaction_type == 'sell':
            transaction_type = 'buy'
            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
            current_transaction_type = None
    elif pnl >= 35:
        if current_transaction_type == 'buy':
            transaction_type = 'sell'
            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
            current_transaction_type = None
        elif current_transaction_type == 'sell':
            transaction_type = 'buy'
            trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=list(latest_candle['price'])[0], current_transaction_type=transaction_type)
            current_transaction_type = None
    
    if timestamp.minute==next_transaction_time.minute and timestamp.second>=next_transaction_time.second:
        
        # Build Candle
        price_list = list(latest_candle['price'])
        if price_list[0]<price_list[-1]:
            color = 'green'
        else:
            color = 'red'
        candle_data.loc[len(candle_data)] = [first_candle_timestamp, price_list[0], max(price_list), min(price_list), price_list[-1], color]
        first_candle_timestamp = first_candle_timestamp+timedelta(minutes=candle)
        latest_candle = pd.DataFrame(columns=['price'])
        
        # Use candle data to enter trade
        if color=='red':
            transaction_type = 'sell'
            if current_status=='open':
                if current_transaction_type == 'buy':
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=price_list[0], current_transaction_type=transaction_type)
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=price_list[0], current_transaction_type=transaction_type)
            elif current_status=='closed':
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=price_list[0], current_transaction_type=transaction_type)
        elif color=='green':
            transaction_type = 'buy'
            if current_status=='open':
                if current_transaction_type == 'sell':
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=price_list[0], current_transaction_type=transaction_type)
                    trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=price_list[0], current_transaction_type=transaction_type)
            elif current_status=='closed':
                trade_log, transaction_price, current_transaction_type, current_status = place_trade(trade_log, current_time=first_candle_timestamp, price=price, open_price=price_list[0], current_transaction_type=transaction_type)
        n=n+1
        next_transaction_time = first_candle_timestamp + timedelta(minutes=candle) - timedelta(seconds=5)
        sleep((first_candle_timestamp-datetime.now()).total_seconds())